# Functions

In [31]:
# Standard library imports
import numpy as np
import pandas as pd

# PyTorch related imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import BCEWithLogitsLoss

# Sklearn related imports
from sklearn.base import BaseEstimator
from sklearn.inspection import permutation_importance
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Adversarial attack related imports
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

# Data visualization imports
import matplotlib.pyplot as plt

# Other imports
import eagerpy as ep
from icecream import ic
from my_operations import XDecode

# Custom module imports (if these are custom modules)
from EspPipeML import esp_utilities


from sklearn.model_selection import StratifiedShuffleSplit
from fastai.tabular.all import *

class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 100)  # First hidden layer with 100 neurons
        self.fc2 = nn.Linear(100, 100)       # Second hidden layer with 100 neurons
        self.fc3 = nn.Linear(100, 100)       # Third hidden layer with 100 neurons
        self.fc4 = nn.Linear(100, 1)         # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))         # Activation for first hidden layer
        x = torch.relu(self.fc2(x))         # Activation for second hidden layer
        x = torch.relu(self.fc3(x))         # Activation for third hidden layer
        x = self.fc4(x)                      # Output layer
        return x


def predict_with_custom_threshold(model, X, threshold=0.4, return_type='pd.Series'):
    """
    Predict labels for input data using a custom classification threshold.

    Parameters:
    model (nn.Module): Trained model for prediction.
    X (pd.DataFrame): Input features.
    threshold (float): Custom threshold for classification.
    return_type (str): Type of the returned variable. Options: 'pd.Series', 'ep_tensor', 'float_tensor'.

    Returns:
    pd.Series or torch.Tensor or eagerpy.tensor.pytorch.PyTorchTensor or original type: Predicted labels.
    """
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        X_tensor = torch.tensor(X.values, dtype=torch.float32).to('cuda')
        outputs = model(X_tensor)

        # specific case when the model is from fastai
        if len(outputs.shape) > 1:
            outputs = outputs[:, 1]

        probabilities = torch.sigmoid(outputs)
        predictions = (probabilities > threshold).float()  # Apply custom threshold
        
        if return_type == 'ep_tensor':
            return ep.astensor(predictions.numpy().flatten())
        elif return_type == 'float_tensor':
            return predictions
        else:
            return pd.Series(predictions.cpu().numpy().flatten())

def predict_with_precomputed_outputs(outputs, threshold=0.4, return_type='pd.Series'):
    """
    Predict labels using precomputed model outputs and a custom classification threshold.

    Parameters:
    outputs (np.ndarray): Precomputed outputs from the model, expected in numpy array format.
    threshold (float): Custom threshold for classification.
    return_type (str): Type of the returned variable. Options: 'pd.Series', 'ep_tensor', 'float_tensor'.

    Returns:
    pd.Series or torch.Tensor or eagerpy.tensor.pytorch.PyTorchTensor or original type: Predicted labels.
    """
    # Convert numpy array to torch tensor and apply sigmoid
    outputs_tensor = torch.from_numpy(outputs).float()
    probabilities = torch.sigmoid(outputs_tensor)
    
    # Apply custom threshold
    predictions = (probabilities > threshold).float()
    
    # Return in the desired format
    if return_type == 'ep_tensor':
        return ep.astensor(predictions.numpy().flatten())
    elif return_type == 'float_tensor':
        return predictions
    else:
        return pd.Series(predictions.numpy().flatten())

def model_evaluation(datasets, model=None, threshold=0.3, detected_attacks_only=False):
    """
    Evaluate the model on given datasets with an option to focus on detected network attacks.

    Parameters:
    datasets (dict): Datasets for evaluation, structured by evasion attack and epsilon values.
    model (nn.Module): The model to be evaluated. If None, precomputed confidences are used.
    threshold (float): Threshold for classification.
    detected_attacks_only (bool): If True, evaluates only on detected network attacks.

    Returns:
    dict: Evasion rates for each attack type and epsilon value.
    """
    if model is None:
        results = {}
        for attack_type in datasets:
            evasion_rates = {}
            for epsilon in datasets[attack_type]:
                df = datasets[attack_type][epsilon].copy(deep=True)
                clean_confidence = df['cleanConfidence'].to_numpy()
                clean_predictions = predict_with_precomputed_outputs(clean_confidence, threshold=threshold)
                df['cleanPred'] = clean_predictions

                if detected_attacks_only:
                    df = df[df['cleanPred'] == 1]

                adv_confidence = df['advConfidence'].to_numpy()
                adv_predictions = predict_with_precomputed_outputs(adv_confidence, threshold=threshold)
                evasion_rate = compute_evasion_rate(np.ones(len(adv_predictions)), adv_predictions)
                evasion_rates[epsilon] = evasion_rate
            results[attack_type] = evasion_rates
        return results

    else:
        # Implement model evaluation using attack data
        pass

def train_mlp(X_train, Y_train, model, num_epochs=100, learning_rate=0.001, batch_size=32, patience=5, val_ratio=0.2):
    """
    Train a multi-layer perceptron (MLP) model with early stopping.

    Parameters:
    X_train (numpy.ndarray): Training features.
    Y_train (numpy.ndarray): Training labels.
    model (nn.Module): MLP model to be trained.
    num_epochs (int): Number of epochs for training.
    learning_rate (float): Learning rate for the optimizer.
    batch_size (int): Batch size for data loading.
    patience (int): Number of epochs to wait for improvement before stopping.
    val_ratio (float): Ratio of training data to be used as validation data.

    Returns:
    None
    """
    # Splitting training data into training and validation sets
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=val_ratio, stratify=Y_train)

    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to('cuda')
    Y_train = torch.tensor(Y_train, dtype=torch.float32).to('cuda')
    X_val = torch.tensor(X_val, dtype=torch.float32).to('cuda')
    Y_val = torch.tensor(Y_val, dtype=torch.float32).to('cuda')

    # Reshape Y if necessary
    Y_train = Y_train.view(Y_train.shape[0], 1)
    Y_val = Y_val.view(Y_val.shape[0], 1)

    # Create datasets and data loaders
    train_dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(X_val, Y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Define loss function and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Early stopping variables
    best_loss = float('inf')
    epochs_no_improve = 0

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, targets).item()

        val_loss /= len(val_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}')

        # Check for early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print("Early stopping triggered")
                break

    print("Training complete!")


def get_network_attacks(datasetloader, network_attack, model=None, threshold=0.4, clip_range=False, feature_set=None):
    """
    Retrieves network attacks from a dataset loader based on the specified network attack category.
    
    Parameters:
        datasetloader (DatasetLoader): The dataset loader object.
        network_attack (str): The network attack category to filter the attacks.
        model (object, optional): The machine learning model used for prediction. Defaults to None.
        threshold (float, optional): The threshold value for prediction. Defaults to 0.4.
        clip_range (bool, optional): Whether to clip the values to 0 and 1. Defaults to False.
        feature_set (list, optional): The list of features to include in the selected attacks. Defaults to None.
    
    Returns:
        dict: A dictionary containing the selected network attacks.
            - 'X' (DataFrame): The features of the selected attacks.
            - 'y' (Series): The labels of the selected attacks.
    """
    # Filter the dataframe to select rows where the label is 1
    f = datasetloader.get_full_test_set()
    filtered_df = f.loc[f['label'] == 1]
    


    # Filter the dataframe to select rows according to network_attack variable e.g. 'DoS'
    filtered_df = filtered_df.loc[filtered_df['attackCategory'] == network_attack]


    # Clip the values to 0 and 1
    if clip_range:
        filtered_df = filtered_df.drop('attackCategory', axis=1).clip(lower=0, upper=1)
    else:
        filtered_df = filtered_df.drop('attackCategory', axis=1)


    # Resetting the index
    filtered_df.reset_index(drop=True, inplace=True)

    # Create a dictionary to store the selected attacks
    selected_attacks = {}
    if feature_set is None:
        selected_attacks['X'] = filtered_df.drop('label', axis=1)
    else:
        selected_attacks['X'] = filtered_df[feature_set]
    selected_attacks['y'] = filtered_df['label']
    
    # Return successfully predicted attacks if model is not None
    if model is not None:
        y_pred = predict_with_custom_threshold(model, selected_attacks['X'], threshold=threshold)

        # Ensure y_pred is a Series and align with selected_attacks['y']
        y_pred = pd.Series(y_pred).reset_index(drop=True)

        # Filtering where predictions match the actual labels
        selected_attacks['X'] = selected_attacks['X'][y_pred == selected_attacks['y']]
        selected_attacks['y'] = selected_attacks['y'][y_pred == selected_attacks['y']]

    return selected_attacks


def compute_evasion_rate(y_true, y_pred):
    """
    Computes the evasion rate for a given set of predictions.
    
    Parameters:
        y_true (Series): The true labels.
        y_pred (Series): The predicted labels.
    
    Returns:
        float: The evasion rate.
    """
    # Compute the number of attacks that were successfully evaded
    num_attacks_evaded = ((y_true == 1) & (y_pred == 0)).sum()
    
    # Compute the number of attacks
    num_attacks = (y_true == 1).sum()
    
    # Compute the evasion rate
    evasion_rate = num_attacks_evaded / num_attacks
    
    return evasion_rate


class PyTorchToSklearnWrapper(BaseEstimator):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        pass

    def predict(self, X):
        # Using the custom prediction function
        return predict_with_custom_threshold(self.model, X, threshold=0.4, return_type='pd.Series')

    def score(self, X, y):
        predictions = self.predict(X)
        # Use 'macro' as an example; you can also try 'micro' or 'weighted'
        return f1_score(y, predictions, average='macro')

# Assuming `model` is your trained PyTorch model


def compute_feature_importance(model, dataset_loader, n_repeats=5, random_state=42, attack_types=None):
    """
    Compute the feature permutation importance for a PyTorch model, with an option to focus on specific network attack types.

    Parameters:
    model (PyTorch model): Trained PyTorch model.
    dataset_loader (DatasetLoader): An instance of DatasetLoader containing the dataset.
    n_repeats (int): Number of times to permute a feature.
    random_state (int): Random state for reproducibility.
    attack_types (list, optional): List of network attack types to include in the analysis.

    Returns:
    pd.DataFrame: DataFrame with feature names and their importance rankings.
    """
    np.random.seed(random_state)  # For reproducibility

    # Create an instance of the wrapper with the trained model
    model_wrapper = PyTorchToSklearnWrapper(model)

    # Extract test dataset
    X_test, y_test, y_test_multiclass = dataset_loader.X_test, dataset_loader.y_test, dataset_loader.y_test_multiclass

    if attack_types is not None:
        # Reset index to ensure alignment
        X_test = X_test.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        y_test_multiclass = y_test_multiclass.reset_index(drop=True)

        # Select instances based on the specified attack types
        attack_indices = y_test_multiclass.isin(attack_types)
        normal_indices = ~attack_indices

        # Balance the dataset with equal number of normal traffic instances
        normal_samples = np.random.choice(X_test[normal_indices].index, sum(attack_indices), replace=False)
        selected_indices = np.concatenate([np.where(attack_indices)[0], normal_samples])

        X_test_selected = X_test.iloc[selected_indices]
        y_test_selected = y_test.iloc[selected_indices]
    else:
        X_test_selected = X_test
        y_test_selected = y_test

    # Calculate permutation importance on the selected test set
    result = permutation_importance(
        model_wrapper, X_test_selected, y_test_selected, 
        n_repeats=n_repeats, random_state=random_state
    )

    # Create a DataFrame with feature names and importances
    importance_df = pd.DataFrame({
        'Feature': X_test_selected.columns,
        'Importance': result.importances_mean
    })

    # Sort the DataFrame by importance
    importance_df.sort_values(by='Importance', ascending=False, inplace=True)
    importance_df.reset_index(drop=True, inplace=True)

    return importance_df


def perform_incremental_adversarial_attacks(model, dataloader, nw_attack_name='DoS', epsilon=0.4, norm=2, pred_threshold=0.4):

    selected_attacks = get_network_attacks(dataloader, network_attack=nw_attack_name, model=None, threshold=None)
    x_selected_attacks = selected_attacks['X'].to_numpy().astype(np.float32)
    y_selected_attacks = selected_attacks['y'].to_numpy().astype(np.float32)
    #y_selected_attacks = y_selected_attacks.reshape(-1, 1)

    n_features = x_selected_attacks.shape[1]
    feature_perturb_order = compute_feature_importance(model, dataloader, attack_types=['DoS'])['Feature'].tolist()
    
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(0, 1),
        loss=nn.BCEWithLogitsLoss(),
        optimizer=optim.Adam(model.parameters(), lr=0.001),
        input_shape=(n_features,),
        nb_classes=2,
    )

    attacks = {
        'FGSM': FastGradientMethod(estimator=classifier, eps=epsilon, norm=norm),
        'PGD': ProjectedGradientDescent(estimator=classifier, eps=epsilon, verbose=True, norm=norm),
    }
    # selecting features to be perturbed
    mask = np.zeros(n_features, dtype=np.float32)

    results = {attack: {} for attack in attacks}
    evasion_rates = {attack: {} for attack in attacks}

    for feature, ttl_perturbed_features in zip(feature_perturb_order, range(1, n_features+1)):
        feature_pos = selected_attacks['X'].columns.get_loc(feature)
        mask[feature_pos] = 1

        # Step 6: Generate adversarial evasion attacks
        for attack_name, attack in attacks.items():

            adv = attack.generate(x=x_selected_attacks, mask=mask)

            # Step 7: Evaluate the ART classifier on adversarial evasion attacks
            predictions_confidense = classifier.predict(adv)
            predicted_classes = (predictions_confidense > pred_threshold).astype(int).ravel()

            evasion_rate = compute_evasion_rate(y_selected_attacks, predicted_classes)
            print("{} Evasion Rate on adversarial examples with {} features: {}%".format(attack_name, ttl_perturbed_features, evasion_rate))

            results[attack_name][ttl_perturbed_features] = adv
            evasion_rates[attack_name][ttl_perturbed_features] = evasion_rate

    return results, evasion_rates

def to_one_hot(targets, n_classes=2, dtype=torch.float32):
    """
    Convert targets to one-hot encoding.

    Parameters:
    targets (pandas.core.series.Series): Array of target labels.
    n_classes (int): Number of classes.

    Returns:
    np array: One-hot encoded targets.
    """
    targets = np.array(targets.astype(int))
    one_hot_targets = np.eye(n_classes)[targets]
    return one_hot_targets.astype(np.float32)

def perform_target_feature_adversarial_attacks(model, dataloader, target_features, nw_attack_name='DoS', epsilons=[0.3, 0.7, 1.0], norm=2, pred_threshold=0.4, fastai_model=False, loss=nn.BCEWithLogitsLoss(), device='cuda', feature_set=None):
    selected_attacks = get_network_attacks(dataloader, network_attack=nw_attack_name, model=None, threshold=None, feature_set=feature_set)
    features = selected_attacks['X'].columns.tolist()
    x_selected_attacks = selected_attacks['X'].to_numpy().astype(np.float32)
    model.to(device)

    y_selected_attacks = selected_attacks['y'].to_numpy().astype(np.float32)
    

    #y_selected_attacks = y_selected_attacks.reshape(-1, 1)

    n_features = x_selected_attacks.shape[1]

    classifier = PyTorchClassifier(
        model=model,
        clip_values=(0, 1),
        loss=loss,
        optimizer=optim.Adam(model.parameters(), lr=0.001),
        input_shape=(n_features,),
        nb_classes=2,
    )

    # selecting features to be perturbed
    mask = np.zeros(n_features, dtype=np.float32)
    for feature in target_features:
        mask[selected_attacks['X'].columns.get_loc(feature)] = 1

    #results = {attack: {} for attack in attacks}
    results = {attack: {epsilon: {} for epsilon in epsilons} for attack in ['FGSM', 'PGD']}
    evasion_rates = {attack: {epsilon: {} for epsilon in epsilons} for attack in ['FGSM', 'PGD']}
    y_logits = classifier.predict(x_selected_attacks)

    if fastai_model:
                y_logits = y_logits[: , 1:2]
    
    y_logits_tensor = torch.from_numpy(y_logits).float()
    y_conf = torch.sigmoid(y_logits_tensor).numpy()
    max_iter = 30
    eps_steps = [x/max_iter for x in epsilons]
    for epsilon, eps_step in zip(epsilons, eps_steps):
        attacks = {
            #'FGSM': FastGradientMethod(estimator=classifier, eps=epsilon, norm=norm, num_random_init=0),
            'FGSM': FastGradientMethod(estimator=classifier, eps=epsilon, norm=norm, num_random_init=0, minimal=True, eps_step=eps_step),
            'PGD': ProjectedGradientDescent(estimator=classifier, eps=epsilon, verbose=True, norm=norm, eps_step=eps_step, num_random_init=0, max_iter=max_iter),
            # 'PGD': ProjectedGradientDescentWithLogging(estimator=classifier, eps=epsilon, verbose=True, norm=norm, eps_step=eps_step, num_random_init=0, max_iter=max_iter),
        }

        # Step 6: Generate adversarial evasion attacks
        for attack_name, attack in attacks.items():

            adv = attack.generate(x=x_selected_attacks, mask=mask)

            # Step 7: Evaluate the ART classifier on adversarial evasion attacks
            
            logits_adv = classifier.predict(adv)

            if fastai_model:
                logits_adv = logits_adv[: , 1:2]
            
            y_conf_adv = torch.sigmoid(torch.from_numpy(logits_adv)).numpy()
                
            # return predictions_confidense_adv
            
            predicted_classes_adv = (y_conf_adv > pred_threshold).astype(int).ravel()

            
            evasion_rate = compute_evasion_rate(y_selected_attacks, predicted_classes_adv)
            print("{} Evasion Rate on adversarial examples with {} epsilon: {}%".format(attack_name, epsilon, evasion_rate))

            df_adv = pd.DataFrame(adv, columns=features)
            df_adv['advConfidence'] = y_conf_adv
            df_adv['cleanConfidence'] = y_conf

            results[attack_name][epsilon] = df_adv
            evasion_rates[attack_name][epsilon] = evasion_rate

    return results, evasion_rates


def plot_evasion_rates(dict1, dict2, names, figsize=(8, 6)):
    """
    Função ajustada para plotar os evasion rates de dois dicionários.
    
    :param dict1: Primeiro dicionário com os dados de evasion rates.
    :param dict2: Segundo dicionário com os dados de evasion rates.
    :param names: Lista com dois elementos, nomes para os conjuntos de dados.
    """
    # Verificando se a lista de nomes tem dois elementos
    if len(names) != 2:
        raise ValueError("A lista de nomes deve conter exatamente dois elementos.")

    # Desempacotando os nomes
    name1, name2 = names

    # Obtendo os valores de epsilon (assumindo que são os mesmos para FGSM e PGD)
    epsilons = list(dict1['FGSM'].keys())

    # Plotando os dados do primeiro dicionário
    plt.figure(figsize=figsize)
    plt.plot(epsilons, list(dict1['FGSM'].values()), label=f'FGSM_{name1}')
    plt.plot(epsilons, list(dict1['PGD'].values()), label=f'PGD_{name1}')

    # Plotando os dados do segundo dicionário
    plt.plot(epsilons, list(dict2['FGSM'].values()), label=f'FGSM_{name2}', linestyle='--')
    plt.plot(epsilons, list(dict2['PGD'].values()), label=f'PGD_{name2}', linestyle='--')

    # Configurações do gráfico
    plt.xticks(epsilons)  # Ajuste para mostrar todos os valores de epsilon no eixo x
    plt.xticks(rotation=90)  # Rotacionando os valores do eixo x
    plt.xlabel('Epsilon')
    plt.ylabel('Evasion Rate')
    plt.title('Evasion Rates for FGSM and PGD Attacks')
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_evasion_rates_features(evasion_dict):
    """
    Função para plotar os evasion rates com o número de features perturbadas no eixo x.
    
    :param evasion_dict: Dicionário com os dados de evasion rates.
    """
    # Plotando os dados
    plt.plot(list(evasion_dict['FGSM'].keys()), list(evasion_dict['FGSM'].values()), label='FGSM')
    plt.plot(list(evasion_dict['PGD'].keys()), list(evasion_dict['PGD'].values()), label='PGD')

    # Configurações do gráfico
    plt.xlabel('Number of Features Perturbed')
    plt.ylabel('Evasion Rate')
    plt.title('Evasion Rates for FGSM and PGD Attacks by Feature Count')
    plt.legend()
    plt.grid(True)
    plt.show()


def f1_score_binary(inputs, targets):
    # Move tensors to CPU and convert to numpy arrays
    inputs = inputs.argmax(dim=1).cpu().numpy()
    targets = targets.cpu().numpy()
    return f1_score(targets, inputs, average='binary', zero_division=0)

def stratified_splitter(df, target, valid_pct=0.2):
    stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=valid_pct, random_state=42)
    for train_index, valid_index in stratified_split.split(df, df[target]):
        return list(train_index), list(valid_index)

def predict_with_custom_threshold_deep(model, X, threshold=0.5, device='cpu'):

    X_tensor = torch.tensor(X.values, dtype=torch.float32).to(device)

    with torch.no_grad():
        model.eval()
        preds = model(X_tensor)[:, 1]

    probs = torch.sigmoid(preds)
    predictions = (probs > threshold).float()

    return pd.Series(predictions.cpu().numpy().flatten())

def predict_with_custom_threshold_deep(model, X, threshold=0.5, device='cpu'):
    # Criar um tensor vazio para dados categóricos
    cat_tensor = torch.tensor([], dtype=torch.int64).reshape(0, 0).to(device)

    # Converter as features contínuas para tensor
    cont_tensor = torch.tensor(X.values, dtype=torch.float32).to(device)

    with torch.no_grad():
        model.eval()
        # Passar ambos os tensores (categórico e contínuo) para o modelo
        preds = model(cat_tensor, cont_tensor)[:, 1]

    probs = torch.sigmoid(preds)
    predictions = (probs > threshold).float()

    return pd.Series(predictions.cpu().numpy().flatten())

# Wrapper para adaptar o modelo FastAI ao ART
class FastAIARTWrapper(torch.nn.Module):
    def __init__(self, fastai_model):
        super(FastAIARTWrapper, self).__init__()
        self.fastai_model = fastai_model.model

    def forward(self, x):
        # x é o tensor de entrada esperado pelo ART
        # x corresponde ao cont_tensor, enquanto cat_tensor é um tensor vazio
        device = next(self.fastai_model.parameters()).device
        cat_tensor = torch.tensor([], dtype=torch.int64).reshape(0, 0).to(device)
        cont_tensor = x.to(device)

        # Passar os tensores pelo modelo FastAI
        preds = self.fastai_model(cat_tensor, cont_tensor)

        return preds#[:, 1] # return the probabilities of the second class
    
def get_ga_best_hyperparameters_and_features(clf_name, metric, data_loader, get_metrics=False, specific_solution=None):
    """
    Get the best hyperparameters and features selected by the genetic algorithm for a given classifier and metric.
    """
    # Load the results from the genetic algorithm
    results = esp_utilities.load_from_pickle(f'../results/nsga2/feature_selection/unsw-nb15_{clf_name}_{metric}_performance_tracker.pkl')

    if specific_solution is None:                            
        best_result = min(results.best_result_per_generation)
        best_gen_id = results.best_result_per_generation.index(best_result)
    else:
        best_gen_id = specific_solution    
    
    if clf_name in ['rf', 'xgb', 'mlp', 'td']:
        n_hp = 2
    else:
        n_hp = 1

    xdecode = XDecode(
        results.best_per_generation[best_gen_id],
        n_hp=n_hp, 
        clf_name=clf_name, 
        feature_names=data_loader.X_train.columns.tolist(), 
        eval=True
    )

    best_hyperparameters_and_features = {'selected_features_names':  xdecode.selected_feature_names}
    

    if clf_name in ['dt', 'rf', 'xgb']:
        threshold = 0.5
        best_hyperparameters_and_features['max_depth'] = int(xdecode.max_depth)

        if clf_name in ['rf', 'xgb']:
            best_hyperparameters_and_features['n_estimators'] = int(xdecode.n_estimators)
    
    if clf_name in ['mlp', 'td']:
        threshold = 0.3
        best_hyperparameters_and_features['n_neurons'] = int(xdecode.n_neurons)
        best_hyperparameters_and_features['n_layers'] = int(xdecode.n_layers)




    if get_metrics:
        metrics = {}
        prob = results.best_y_pred_per_generation[0]
        y_pred = np.where(prob > threshold, 1, 0)
        metrics['attackwise_accuracy'] = esp_utilities.compute_attackwise_accuracy(results.y_true['label'], y_pred, results.y_multi_class['attackCategory'])
        metrics['f1_score'] = f1_score(results.y_true['label'], y_pred, average='weighted')
        metrics['auc'] = roc_auc_score(results.y_true['label'], prob)
        metrics['best_gen'] = best_gen_id
        best_hyperparameters_and_features['metrics'] = metrics
        # fig, auc_score = plot_roc_curve(results.y_true['label'], prob, title=f'ROC Curve for {clf_name.upper()} - {metric.upper()}')
    return best_hyperparameters_and_features

from art.summary_writer import SummaryWriter

import logging
logger = logging.getLogger(__name__)

class ProjectedGradientDescentWithLogging(ProjectedGradientDescent):
    """
    Extended Projected Gradient Descent with logging for the number of iterations.
    """

    def __init__(
        self,
        estimator: Union["CLASSIFIER_LOSS_GRADIENTS_TYPE", "OBJECT_DETECTOR_TYPE"],
        norm: Union[int, float, str] = np.inf,
        eps: Union[int, float, np.ndarray] = 0.3,
        eps_step: Union[int, float, np.ndarray] = 0.1,
        decay: Optional[float] = None,
        max_iter: int = 100,
        targeted: bool = False,
        num_random_init: int = 0,
        batch_size: int = 32,
        random_eps: bool = False,
        summary_writer: Union[str, bool, SummaryWriter] = False,
        verbose: bool = True,
    ):
        super().__init__(
            estimator=estimator,
            norm=norm,
            eps=eps,
            eps_step=eps_step,
            decay=decay,
            max_iter=max_iter,
            targeted=targeted,
            num_random_init=num_random_init,
            batch_size=batch_size,
            random_eps=random_eps,
            summary_writer=summary_writer,
            verbose=verbose,
        )
        self.iteration_count = 0

    def generate(self, x: np.ndarray, y: Optional[np.ndarray] = None, **kwargs) -> np.ndarray:
        self.iteration_count = 0
        logger.info("Creating adversarial samples.")
        adv_x = super().generate(x=x, y=y, **kwargs)
        logger.info(f"Total number of iterations: {self.iteration_count}")
        return adv_x

    def _generate_batch(self, x_batch: np.ndarray, y_batch: np.ndarray) -> np.ndarray:
        for i in range(self.max_iter):
            self.iteration_count += 1
            logger.info(f"Iteration {self.iteration_count}/{self.max_iter}")
            # Perform one iteration of the attack
            x_batch = self._attack._generate_batch(x_batch, y_batch)
        return x_batch

## Loading variables

In [2]:
esp_utilities.fix_seeds(42)
unsw_nb15 = esp_utilities.DatasetLoader('unsw-nb15', scale_data=True, scale_on_full_dataset=False)
time_based_features = ['activeTimeMaxMilliseconds', 'activeTimeMeanMilliseconds', 'interPacketTimeMillisecondsStdev', 'interPacketTimeSecondsStdev', 'idleMeanMilliseconds', 'interPacketTimeSecondsSumFwd', 'interPacketTimeMillisecondsMax', 'idleMaxMilliseconds', 'interPacketTimeSecondsMax', 'activeTimeStdMilliseconds', 'interPacketTimeMillisecondsSum', 'interPacketTimeSecondsMean', 'interPacketTimeSecondsStdevFwd', 'flowDurationMilliseconds', 'idleStdMilliseconds', 'interPacketTimeMillisecondsSumFwd', 'interPacketTimeSecondsMaxFwd', 'interPacketTimeSecondsMeanFwd', 'interPacketTimeSecondsMaxBwd', 'interPacketTimeSecondsStdevBwd', 'interPacketTimeSecondsMeanBwd', 'interPacketTimeMilliseconds', 'bwdJitterMilliseconds', 'interPacketTimeSecondsSumBwd']
volume_based_features = ['minimumIpTotalLength', 'minimumIpTotalLengthFwd', 'minimumIpTotalLengthBwd', 'bwdBytesAvg', 'fwdBytesAvg', 'ipTotalLengthMeanBwd', 'ipTotalLengthMeanFwd', 'packetLenAvg', 'ipTotalLengthMean', 'maximumIpTotalLengthFwd', 'maximumIpTotalLength', 'maximumIpTotalLengthBwd', 'ipTotalLengthStdevBwd', 'octetTotalCountFwd', 'ipTotalLengthFwd', 'ipTotalLengthBwd', 'octetTotalCountBwd', 'packetLen', 'ipTotalLength', 'octetTotalCount', 'ipTotalLengthStdevFwd', 'bwdBytesPerMicroseconds', 'ipTotalLengthVar', 'ipTotalLengthStdev', 'flowBytesByMicroseconds']
mixed_features = list(set(time_based_features+volume_based_features))


In [3]:
DEVICE = 'cuda:0'
LAYERS = [100,100]
THRESHOLD_LIMIAR = 0.3

# Preparing models

### Train MLP model and save it

In [4]:
def train_mlp_model_and_save_it(data_loader, input_dim, num_layers=3, num_neuron=100, device=DEVICE, ga=False, feature_set=None):
    # Load the dataset
    unsw_nb15 = data_loader

    # Create an instance of the MLP model
    if ga:
        model = esp_utilities.MLP(
        input_dim=input_dim, 
        num_neurons=num_neuron, 
        num_layers=num_layers
        )
    else:
        model = MLP(input_dim)
    
    model.to(device)
    model.to(device)

    # Train the MLP model
    if feature_set is None:
        train_mlp(unsw_nb15.X_train.to_numpy(), unsw_nb15.y_train.to_numpy(), model)
    else:
        train_mlp(unsw_nb15.X_train[feature_set].to_numpy(), unsw_nb15.y_train.to_numpy(), model)

    # Save the trained model
    if ga:
        torch.save(model.state_dict(), 'models/mlp_pytorch_ga_usnw-nb15.pth')
    else:
        torch.save(model.state_dict(), 'models/mlp_pytorch_default_usnw-nb15.pth')
    return model

# model = train_mlp_model_and_save_it(data_loader=unsw_nb15, input_dim = unsw_nb15.X_train.shape[1])
# mlp_ga_inf = get_ga_best_hyperparameters_and_features('mlp', 'auc', data_loader=unsw_nb15, get_metrics=True)
# mlp_ga_layer = mlp_ga_inf['n_layers']
# mlp_ga_neuron = mlp_ga_inf['n_neurons']
# mlp_ga_feature_set = mlp_ga_inf['selected_features_names']
# mlp_ga_input_dim = len(mlp_ga_feature_set)
# train_mlp_model_and_save_it(data_loader=unsw_nb15, num_layers=mlp_ga_layer, num_neuron=mlp_ga_neuron, device=DEVICE, ga=True, input_dim=mlp_ga_input_dim, feature_set=mlp_ga_feature_set)

### Load trained MLP model

In [5]:
def load_trained_mlp_model(input_dim, num_layers=3, num_neuron=100, device=DEVICE, ga=False):
    # Create an instance of the MLP model
    if ga:
        model = esp_utilities.MLP(
        input_dim=input_dim, 
        num_neurons=num_neuron, 
        num_layers=num_layers
        )
    else:
        model = MLP(input_dim)
    
    # model.to(device)

    # Load the pre-trained model weights
    if ga:
        model.load_state_dict(torch.load('models/mlp_pytorch_ga_usnw-nb15.pth'))
    else:
        model.load_state_dict(torch.load('models/mlp_pytorch_default_usnw-nb15.pth'))

    # Set the model to evaluation mode
    model.eval()
    model.to(device)

    return model

# No feature selection
mlp_model = load_trained_mlp_model(input_dim=unsw_nb15.X_train.shape[1], device=DEVICE)


# Feature selection (GA)
mlp_ga_inf = get_ga_best_hyperparameters_and_features('mlp', 'auc', data_loader=unsw_nb15, get_metrics=True)
mlp_ga_layer = mlp_ga_inf['n_layers']
mlp_ga_neuron = mlp_ga_inf['n_neurons']
mlp_ga_feature_set = mlp_ga_inf['selected_features_names']
mlp_ga_input_dim = len(mlp_ga_feature_set)
mlp_ga_model = esp_utilities.load_trained_mlp_model(input_dim=mlp_ga_input_dim, device=DEVICE, ga=True, num_layers=mlp_ga_layer, num_neuron=mlp_ga_neuron)

Carregando modelo de: /mnt/nvme1n1/aespindola/quinta_data_dump/Playground0108/source/models/mlp_pytorch_ga_usnw-nb15.pth


In [6]:
# mlp_model = esp_utilities.load_trained_mlp_model(input_dim=unsw_nb15.X_train.shape[1], device=DEVICE)
# mlp_ga_model = esp_utilities.load_trained_mlp_model(input_dim=mlp_ga_input_dim, device=DEVICE, ga=True, num_layers=mlp_ga_layer, num_neuron=mlp_ga_neuron)

### Train Tabular Deep Model and save it 

In [6]:
def train_tabular_deep_model_and_save_it(loader = None, layers=LAYERS, device=DEVICE, n_layers=None, ga=False, feature_set=None):
    # Load the dataset
    if loader is None:
        unsw_nb15 = esp_utilities.DatasetLoader('unsw-nb15', scale_data=True, scale_on_full_dataset=False)
    else:
        unsw_nb15 = loader

    # Combine the features and target variable into single DataFrames for training and testing
    if feature_set is None:
        train_df = pd.concat([unsw_nb15.X_train, unsw_nb15.y_train], axis=1)
        test_df = pd.concat([unsw_nb15.X_test, unsw_nb15.y_test], axis=1)
        # Define your continuous and categorical features
        # Since all your features are numerical, you will only have continuous features
        cont_names = unsw_nb15.X_train.columns.tolist()  # Assuming X_train is a DataFrame
    else:
        train_df = pd.concat([unsw_nb15.X_train[feature_set], unsw_nb15.y_train], axis=1)
        test_df = pd.concat([unsw_nb15.X_test[feature_set], unsw_nb15.y_test], axis=1)
        cont_names = feature_set

    train_df['label'] = train_df['label'].astype(float)
    test_df['label'] = test_df['label'].astype(float)



    
    cat_names = []  # No categorical data in your case

    # Define your target variable
    y_name = 'label'  # Replace 'target' with the name of your target variable in y_train

    # Define preprocessing steps using TabularPandas (if any)
    procs = []  # No additional preprocessing needed as the data is already normalized


    splits = stratified_splitter(train_df, y_name, valid_pct=0.2)
    to = TabularPandas(train_df, 
                    procs=procs, 
                    cat_names=cat_names, 
                    cont_names=cont_names, 
                    y_names=y_name, 
                    splits=splits, 
                    y_block=CategoryBlock())  # Use CategoryBlock for classification tasks


    # Create DataLoaders with specified device
    dls = to.dataloaders(bs=256, device=torch.device(device))


    # Create and move the model to the specified device
    learn = tabular_learner(dls, layers=layers, metrics=f1_score_binary)
    # learn = tabular_learner(dls, layers=layers, metrics=f1_score_binary, y_range=(0,1), n_out=1, loss_func=BCEWithLogitsLoss())
    learn.model.to(torch.device(device))


     # Train the model
    learn.fit_one_cycle(100, 0.001, cbs=[EarlyStoppingCallback(monitor='valid_loss', min_delta=0, patience=5)])
    learn.model_dir = 'models'
    if ga:
        learn.save('deep_'+str(n_layers)+'_fastai_ga_usnw-nb15')
    else:
        learn.save('deep_'+str(n_layers)+'_fastai_default_usnw-nb15')
    return learn, test_df

# Baseline model without feature selection
# tabular_model, d = train_tabular_deep_model_and_save_it(loader=unsw_nb15, device=DEVICE, n_layers='5_512', layers=[512]*5)

# Baseline model with feature selection
# td_ga_inf = get_ga_best_hyperparameters_and_features('td', 'auc', data_loader=unsw_nb15, get_metrics=True)
# td_ga_layer = td_ga_inf['n_layers']
# td_ga_neuron = td_ga_inf['n_neurons']
# n_layer_name = str(td_ga_layer) + '_' + str(td_ga_neuron) # e.g. '5_512'
# td_ga_feature_set = td_ga_inf['selected_features_names']
# # train_tabular_deep_model_and_save_it(loader=unsw_nb15, device=DEVICE, n_layers=n_layer_name, layers=[td_ga_neuron]*td_ga_layer, ga=True, feature_set=td_ga_feature_set)
# ic(td_ga_layer, td_ga_neuron, n_layer_name)

### Load trained Tabular Deep Model

In [7]:

def load_trained_tabular_deep_model(loader, layers, path='deep_fastai_default_usnw-nb15', device=DEVICE, feature_set=None):
    train_df = pd.concat([loader.X_train, loader.y_train], axis=1)

    # Define continuous and categorical columns
    if feature_set is None:
        cont_names = loader.X_train.columns.tolist()  # All features are continuous
    else:
        cont_names = feature_set
    cat_names = []  # Empty list for categorical features, as we only have continuous features

    # Define the target variable
    y_name = 'label'

    # Create TabularPandas object
    procs = [] 
    splits = stratified_splitter(train_df, y_name, valid_pct=0.2)  # Split data for training and validation
    to = TabularPandas(train_df, procs=procs, cat_names=cat_names, cont_names=cont_names, y_names=y_name, splits=splits, y_block=CategoryBlock())

    # Create DataLoaders
    dls = to.dataloaders(bs=256, device=torch.device(device))

    # Create Learner and load the model
    learn = tabular_learner(dls, layers=layers, metrics=f1_score_binary)
    learn.load(path)  # Load the saved model

    return learn

# Baseline model without feature selection
tabular_model = load_trained_tabular_deep_model(loader=unsw_nb15, layers=[512]*5, path='deep_5_512_fastai_default_usnw-nb15', device=DEVICE)
wrapped_tabular_model = FastAIARTWrapper(tabular_model)




# Baseline model with feature selection
td_ga_inf = get_ga_best_hyperparameters_and_features('td', 'auc', data_loader=unsw_nb15, get_metrics=True)
# td_ga_inf = get_ga_best_hyperparameters_and_features('td', 'auc', data_loader=unsw_nb15, get_metrics=True, specific_solution=8)
td_ga_layer = td_ga_inf['n_layers']
td_ga_neuron = td_ga_inf['n_neurons']
n_layer_name = str(td_ga_layer) + '_' + str(td_ga_neuron) # e.g. '5_512'
td_ga_feature_set = td_ga_inf['selected_features_names']
# td_ga_input_dim = len(td_ga_feature_set)
tabular_ga_model = load_trained_tabular_deep_model(loader=unsw_nb15, layers=[td_ga_neuron]*td_ga_layer, path='deep_' + n_layer_name + '_fastai_ga_usnw-nb15', device=DEVICE,  feature_set=td_ga_feature_set)
wrapped_tabular_ga_model = FastAIARTWrapper(tabular_ga_model)

# Adversarial Evasion

In [26]:
# def run_evasion_attacks(model, nw_attacks, eps, data_loader=unsw_nb15, device=DEVICE, fastai_model=False, ga=False, threshold_limiar=THRESHOLD_LIMIAR, extra_mark=None):
def run_evasion_attacks(model, nw_attacks, eps, data_loader=unsw_nb15, device=DEVICE, fastai_model=False, ga=False, threshold_limiar=THRESHOLD_LIMIAR, extra_mark=None, feature_set=None):
    for features_based, feature_set_name in zip([time_based_features, volume_based_features, mixed_features], 
                                            ['time_based_features', 'volume_based_features', 'mixed_features']):
        if feature_set is not None:
            # Intersect the selected features with the current feature set
            features_based = list(set(features_based) & set(feature_set))
            
        for nw_attack in nw_attacks:
            ic(fastai_model, feature_set_name, nw_attack)
            if fastai_model:
                if ga:
                    # path = '../datasets/unrealistic_attacks/unsw-nb15_tabular_deep_ga_fastai_adv_exp_target_' + nw_attack + '_' + feature_set_name + extra_mark +'.pkl'
                    path = '../datasets/unrealistic_attacks/unsw-nb15_tabular_deep_ga_fastai_adv_exp_target_' + nw_attack + '_' + feature_set_name + '.pkl'
                else:
                    path = '../datasets/unrealistic_attacks/unsw-nb15_tabular_deep_5_512_fastai_adv_exp_target_' + nw_attack + '_' + feature_set_name + '.pkl'
                loss = nn.CrossEntropyLoss()
            else:
                if ga:
                    path = '../datasets/unrealistic_attacks/unsw-nb15_mlp_ga_adv_exp_target_' + nw_attack + '_' + feature_set_name + '.pkl'
                else:
                    path = '../datasets/unrealistic_attacks/unsw-nb15_mlp_adv_exp_target_' + nw_attack + '_' + feature_set_name + '.pkl'
                loss = nn.BCEWithLogitsLoss()
            
            adv_exp_target_volume, evasion_rates_volume = perform_target_feature_adversarial_attacks(
                model, 
                data_loader, 
                target_features=features_based, 
                nw_attack_name=nw_attack, 
                epsilons=eps, 
                norm=np.inf, 
                pred_threshold=threshold_limiar,
                fastai_model=fastai_model,
                device=device,
                loss=loss,
                feature_set=feature_set,
                )
            
            esp_utilities.save_to_pickle(adv_exp_target_volume, path)

### Variables

In [10]:
# extra_mark = '_ep0_2to1'
# nw_attacks =['DoS']
# eps = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# run_evasion_attacks(model=mlp_model, nw_attacks=nw_attacks, eps=eps, fastai_model=False, extra_mark=extra_mark)

In [11]:
# nw_attacks =['DoS', 'Exploits', 'Fuzzers', 'Reconnaissance', 'Generic', 'Shellcode', 'Analysis', 'Backdoors', 'Worms']
nw_attacks =['DoS']
eps = [0.00001, 0.0001, 0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

### Crafting Adversarial Evasion for MLP Model and Tabular Deep Model using DoS Attacks

In [32]:
run_evasion_attacks(model=mlp_model, nw_attacks=nw_attacks, eps=eps, fastai_model=False) #18m 46s

ic| fastai_model: False
    feature_set_name: 'time_based_features'
    nw_attack: 'DoS'


FGSM Evasion Rate on adversarial examples with 1e-05 epsilon: 0.03018027141989062%


PGD Evasion Rate on adversarial examples with 1e-05 epsilon: 0.03018027141989062%
FGSM Evasion Rate on adversarial examples with 0.0001 epsilon: 0.032813449463236784%


PGD Evasion Rate on adversarial examples with 0.0001 epsilon: 0.033218553777597734%
FGSM Evasion Rate on adversarial examples with 0.001 epsilon: 0.3609479440956046%


PGD Evasion Rate on adversarial examples with 0.001 epsilon: 0.36155560056714603%
FGSM Evasion Rate on adversarial examples with 0.005 epsilon: 0.43244885558031193%


...

PGD Evasion Rate on adversarial examples with 0.06 epsilon: 0.7569374113834312%
FGSM Evasion Rate on adversarial examples with 0.07 epsilon: 0.6584970629937209%


PGD Evasion Rate on adversarial examples with 0.07 epsilon: 0.7721288231719667%
FGSM Evasion Rate on adversarial examples with 0.08 epsilon: 0.6291270002025522%


PGD Evasion Rate on adversarial examples with 0.08 epsilon: 0.7885355479035852%
FGSM Evasion Rate on adversarial examples with 0.09 epsilon: 0.6173789750860846%


PGD Evasion Rate on adversarial examples with 0.09 epsilon: 0.8035244075349403%
FGSM Evasion Rate on adversarial examples with 0.1 epsilon: 0.61109985821349%


PGD Evasion Rate on adversarial examples with 0.1 epsilon: 0.8142596718655054%


In [33]:
# MLP GA (with feature selection)
run_evasion_attacks(model=mlp_ga_model, nw_attacks=nw_attacks, eps=eps, fastai_model=False, ga=True, feature_set=mlp_ga_feature_set) # 13m 30s 

ic| fastai_model: False
    feature_set_name: 'time_based_features'
    nw_attack: 'DoS'


FGSM Evasion Rate on adversarial examples with 1e-05 epsilon: 0.01843224630342313%


PGD Evasion Rate on adversarial examples with 1e-05 epsilon: 0.01843224630342313%
FGSM Evasion Rate on adversarial examples with 0.0001 epsilon: 0.020255215718047396%


PGD Evasion Rate on adversarial examples with 0.0001 epsilon: 0.020255215718047396%
FGSM Evasion Rate on adversarial examples with 0.001 epsilon: 0.09580717034636418%


PGD Evasion Rate on adversarial examples with 0.001 epsilon: 0.14766052258456552%
FGSM Evasion Rate on adversarial examples with 0.005 epsilon: 0.6846262912700021%


...

PGD Evasion Rate on adversarial examples with 0.06 epsilon: 0.9898723921409763%
FGSM Evasion Rate on adversarial examples with 0.07 epsilon: 0.8849503747214907%


PGD Evasion Rate on adversarial examples with 0.07 epsilon: 0.9898723921409763%
FGSM Evasion Rate on adversarial examples with 0.08 epsilon: 0.8857605833502127%


PGD Evasion Rate on adversarial examples with 0.08 epsilon: 0.9904800486125177%
FGSM Evasion Rate on adversarial examples with 0.09 epsilon: 0.8979137127810412%


PGD Evasion Rate on adversarial examples with 0.09 epsilon: 0.9902774964553372%
FGSM Evasion Rate on adversarial examples with 0.1 epsilon: 0.9096617378975086%


PGD Evasion Rate on adversarial examples with 0.1 epsilon: 0.9912902572412396%


In [34]:
run_evasion_attacks(model=wrapped_tabular_model, nw_attacks=nw_attacks, eps=eps, fastai_model=True)

ic| fastai_model: True
    feature_set_name: 'time_based_features'
    nw_attack: 'DoS'


FGSM Evasion Rate on adversarial examples with 1e-05 epsilon: 0.0435487137938019%


PGD Evasion Rate on adversarial examples with 1e-05 epsilon: 0.0435487137938019%
FGSM Evasion Rate on adversarial examples with 0.0001 epsilon: 0.038079805549929104%


PGD Evasion Rate on adversarial examples with 0.0001 epsilon: 0.038079805549929104%
FGSM Evasion Rate on adversarial examples with 0.001 epsilon: 0.02349605023293498%


PGD Evasion Rate on adversarial examples with 0.001 epsilon: 0.3593275268381608%
FGSM Evasion Rate on adversarial examples with 0.005 epsilon: 0.03159813652015394%


...

PGD Evasion Rate on adversarial examples with 0.06 epsilon: 0.5460806157585578%
FGSM Evasion Rate on adversarial examples with 0.07 epsilon: 0.5122544055094187%


PGD Evasion Rate on adversarial examples with 0.07 epsilon: 0.5460806157585578%
FGSM Evasion Rate on adversarial examples with 0.08 epsilon: 0.5260279521976909%


PGD Evasion Rate on adversarial examples with 0.08 epsilon: 0.5460806157585578%
FGSM Evasion Rate on adversarial examples with 0.09 epsilon: 0.5369657686854366%


PGD Evasion Rate on adversarial examples with 0.09 epsilon: 0.5460806157585578%
FGSM Evasion Rate on adversarial examples with 0.1 epsilon: 0.5442576463439336%


PGD Evasion Rate on adversarial examples with 0.1 epsilon: 0.5460806157585578%


In [35]:
# Tabular deep GA (with feature selection)
run_evasion_attacks(model=wrapped_tabular_ga_model, nw_attacks=nw_attacks, eps=eps, fastai_model=True, ga=True, feature_set=td_ga_feature_set) # 54 minutes

ic| fastai_model: True
    feature_set_name: 'time_based_features'
    nw_attack: 'DoS'


FGSM Evasion Rate on adversarial examples with 1e-05 epsilon: 0.0016204172574437918%


PGD Evasion Rate on adversarial examples with 1e-05 epsilon: 0.0016204172574437918%
FGSM Evasion Rate on adversarial examples with 0.0001 epsilon: 0.0010127607859023698%


PGD Evasion Rate on adversarial examples with 0.0001 epsilon: 0.0010127607859023698%
FGSM Evasion Rate on adversarial examples with 0.001 epsilon: 0.0012153129430828439%


PGD Evasion Rate on adversarial examples with 0.001 epsilon: 0.1409763013976099%
FGSM Evasion Rate on adversarial examples with 0.005 epsilon: 0.027141989062183514%


...

PGD Evasion Rate on adversarial examples with 0.06 epsilon: 0.5643103099048005%
FGSM Evasion Rate on adversarial examples with 0.07 epsilon: 0.38869758962932954%


PGD Evasion Rate on adversarial examples with 0.07 epsilon: 0.5647154142191614%
FGSM Evasion Rate on adversarial examples with 0.08 epsilon: 0.42353656066437106%


PGD Evasion Rate on adversarial examples with 0.08 epsilon: 0.5645128620619809%
FGSM Evasion Rate on adversarial examples with 0.09 epsilon: 0.43933562892444805%


PGD Evasion Rate on adversarial examples with 0.09 epsilon: 0.5647154142191614%
FGSM Evasion Rate on adversarial examples with 0.1 epsilon: 0.4466275065829451%


PGD Evasion Rate on adversarial examples with 0.1 epsilon: 0.5651205185335224%


### Crafting Adversarial Evasion for MLP Model and Tabular Deep Model using all remaining Attacks

In [8]:
nw_attacks =['Exploits', 'Fuzzers', 'Reconnaissance', 'Generic', 'Shellcode', 'Analysis', 'Backdoors', 'Worms']

In [12]:
run_evasion_attacks(model=mlp_model, nw_attacks=nw_attacks, eps=eps, fastai_model=False) # 946 minutes

ic| fastai_model: False
    feature_set_name: 'time_based_features'
    nw_attack: 'DoS'


FGSM Evasion Rate on adversarial examples with 1e-05 epsilon: 0.03018027141989062%


PGD Evasion Rate on adversarial examples with 1e-05 epsilon: 0.03018027141989062%
FGSM Evasion Rate on adversarial examples with 0.0001 epsilon: 0.03018027141989062%


PGD Evasion Rate on adversarial examples with 0.0001 epsilon: 0.03018027141989062%
FGSM Evasion Rate on adversarial examples with 0.001 epsilon: 0.03018027141989062%


PGD Evasion Rate on adversarial examples with 0.001 epsilon: 0.038890014178651004%
FGSM Evasion Rate on adversarial examples with 0.005 epsilon: 0.03018027141989062%


...

PGD Evasion Rate on adversarial examples with 0.06 epsilon: 0.6254810613733036%
FGSM Evasion Rate on adversarial examples with 0.07 epsilon: 0.03018027141989062%


PGD Evasion Rate on adversarial examples with 0.07 epsilon: 0.6129228276281142%
FGSM Evasion Rate on adversarial examples with 0.08 epsilon: 0.03018027141989062%


PGD Evasion Rate on adversarial examples with 0.08 epsilon: 0.6305448653028155%
FGSM Evasion Rate on adversarial examples with 0.09 epsilon: 0.03018027141989062%


PGD Evasion Rate on adversarial examples with 0.09 epsilon: 0.646141381405712%
FGSM Evasion Rate on adversarial examples with 0.1 epsilon: 0.03018027141989062%


PGD Evasion Rate on adversarial examples with 0.1 epsilon: 0.6724731618391736%


In [ ]:
# MLP GA (with feature selection)
run_evasion_attacks(model=mlp_ga_model, nw_attacks=nw_attacks, eps=eps, fastai_model=False, ga=True) # 251 minutes

In [ ]:
run_evasion_attacks(model=wrapped_tabular_model, nw_attacks=nw_attacks, eps=eps, fastai_model=True)

ic| fastai_model: True
    feature_set_name: 'time_based_features'
    nw_attack: 'Exploits'


FGSM Evasion Rate on adversarial examples with 1e-05 epsilon: 0.03299997347269013%


PGD Evasion Rate on adversarial examples with 1e-05 epsilon: 0.03310608271215216%
FGSM Evasion Rate on adversarial examples with 0.0001 epsilon: 0.030108496697349923%


PGD Evasion Rate on adversarial examples with 0.0001 epsilon: 0.031355280261028726%
FGSM Evasion Rate on adversarial examples with 0.001 epsilon: 0.08706263097859246%


PGD Evasion Rate on adversarial examples with 0.001 epsilon: 0.13112449266519882%
FGSM Evasion Rate on adversarial examples with 0.005 epsilon: 0.8037774889248481%


...

PGD Evasion Rate on adversarial examples with 0.06 epsilon: 0.9894736842105263%
FGSM Evasion Rate on adversarial examples with 0.07 epsilon: 0.9368421052631579%


PGD Evasion Rate on adversarial examples with 0.07 epsilon: 0.9947368421052631%
FGSM Evasion Rate on adversarial examples with 0.08 epsilon: 0.9473684210526315%


PGD Evasion Rate on adversarial examples with 0.08 epsilon: 0.9947368421052631%
FGSM Evasion Rate on adversarial examples with 0.09 epsilon: 0.9578947368421052%


PGD Evasion Rate on adversarial examples with 0.09 epsilon: 0.9947368421052631%
FGSM Evasion Rate on adversarial examples with 0.1 epsilon: 0.968421052631579%


PGD Evasion Rate on adversarial examples with 0.1 epsilon: 0.9894736842105263%


In [ ]:
# Tabular deep GA (with feature selection)
run_evasion_attacks(model=wrapped_tabular_ga_model, nw_attacks=nw_attacks, eps=eps, fastai_model=True, ga=True) # 731 minutes

ic| fastai_model: True
    feature_set_name: 'time_based_features'
    nw_attack: 'Exploits'


FGSM Evasion Rate on adversarial examples with 1e-05 epsilon: 0.0034220229726503433%


PGD Evasion Rate on adversarial examples with 1e-05 epsilon: 0.0034220229726503433%
FGSM Evasion Rate on adversarial examples with 0.0001 epsilon: 0.003289386423322811%


PGD Evasion Rate on adversarial examples with 0.0001 epsilon: 0.0033159137331883175%
FGSM Evasion Rate on adversarial examples with 0.001 epsilon: 0.08295089794943895%


PGD Evasion Rate on adversarial examples with 0.001 epsilon: 0.08332228028755603%
FGSM Evasion Rate on adversarial examples with 0.005 epsilon: 0.38156882510544604%


...

PGD Evasion Rate on adversarial examples with 0.06 epsilon: 0.9368421052631579%
FGSM Evasion Rate on adversarial examples with 0.07 epsilon: 0.7421052631578947%


PGD Evasion Rate on adversarial examples with 0.07 epsilon: 0.9368421052631579%
FGSM Evasion Rate on adversarial examples with 0.08 epsilon: 0.7578947368421053%


PGD Evasion Rate on adversarial examples with 0.08 epsilon: 0.9421052631578948%
FGSM Evasion Rate on adversarial examples with 0.09 epsilon: 0.7631578947368421%


PGD Evasion Rate on adversarial examples with 0.09 epsilon: 0.9473684210526315%
FGSM Evasion Rate on adversarial examples with 0.1 epsilon: 0.7736842105263158%


PGD Evasion Rate on adversarial examples with 0.1 epsilon: 0.9631578947368421%
